# <p style="text-align: center;"> Variational Linear Systems Code </p> 
<p style="text-align: center;"> Ryan LaRose </p>

This notebook briefly demonstrates the current state of the Variational Linear Systems (VLS) code. All code is contained in `vls_pauli.py`, which defines a `PauliSystem` class.

In [1]:
# =======
# imports
# =======
import time

import numpy as np

from vls_pauli import PauliSystem
from cirq import ParamResolver

# Creating a Linear System of Equations

A `PauliSystem` consists of a matrix of the form
\begin{equation}
A = \sum_{k = 1}^{K} c_k \sigma_k
\end{equation}
where $c_k$ are complex coefficients and $\sigma_k$ are strings of Pauli operators. In code, we represent the matrix $A$ as arrays of strings corresponding to Pauli operators. For example, to represent the Pauli operators
\begin{align}
\sigma_1 &= \sigma_X \otimes \sigma_Y \otimes \sigma_X \otimes \sigma_Z \\
\sigma_2 &= \sigma_Z \otimes \sigma_X \otimes \sigma_I \otimes \sigma_Y
\end{align}
we would write:

In [2]:
# specify the pauli operators of the matrix
Amat_ops = np.array([["X", "Y", "X", "Z"],
                     ["Z", "X", "I", "Y"]])

Coefficients $c_k$ are stored similarly as arrays of complex values:

In [3]:
# specify the coefficients of each term in the matrix
Amat_coeffs = np.array([0.5 + 0.5j, 1 - 2j])

Finally, the solution vector
\begin{equation}
|b\rangle = U |0\>
\end{equation}
is represented by the unitary $U$ that (efficiently) prepares $|b\rangle$ from the ground state. For example, the unitary $U$ could be
\begin{equation}
U = \sigma_x \otimes \sigma_x \otimes \sigma_I \otimes \sigma_X,
\end{equation}
which we would represent in code as:

In [4]:
# specify the unitary that prepares the solution vector b
Umat_ops = np.array(["X", "X", "I", "X"])

To create `PauliSystem`, we can then simply feed in `Amat_coeffs`, `Amat_ops`, and `Umat_ops`.

In [5]:
# create a linear system of equations
system = PauliSystem(Amat_coeffs, Amat_ops, Umat_ops)

# Working with a `PauliSystem`

The `PauliSystem` class can tell basic information about the system:

In [6]:
print("Number of qubits in system:", system.num_qubits())
print("Size of matrix:", system.size())

Number of qubits in system: 4
Size of matrix: (16, 16)


To see the actual matrix representation of the system (in the computational basis), we can do:

In [7]:
# get the matrix of the system
matrix = system.matrix()
print(matrix)

[[ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j  -2. -1.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0.5-0.5j  0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   2. +1.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j  -0.5+0.5j]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
  -2. -1.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0.5-0.5j  0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   2. +1.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j  -0.5+0.5j
   0. +0.j   0. +0.j ]
 [ 0. +0.j  -2. -1.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j  -0.5+0.5j  0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 2. +1.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0.5-0.5j  0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.

We can also see the solution vector $|b\rangle$ by doing:

In [8]:
b = system.vector()
print(b)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]


# Creating an Ansatz

Initially, the `PauliSystem` ansatz for $V$ is an empty circuit:

In [9]:
# print out the initial (empty) ansatz
system.ansatz

We are free to pick whatever ansatz we wish. The method `PauliSystem.make_ansatz_circuit` is currently set to make an alternating two-qubit gate ansatz.

In [10]:
# make the alternating two-qubit gate ansatz and print it out
system.make_ansatz_circuit()
system.ansatz # note: circuits aren't printed to allow for scrollable outputs

1: ───X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")────Y^Symbol("7")────Z^Symbol("8")────@───X^Symbol("39")───Y^Symbol("40")───Z^Symbol("41")───────X───X^Symbol("45")───Y^Symbol("46")───Z^Symbol("47")───────X───
                                                         │                                                      │                                                          │                                                          │
2: ───X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")────Y^Symbol("10")───Z^Symbol("11")───X───X^Symbol("24")───Y^Symbol("25")───Z^Symbol("26")───@───┼───X^Symbol("30")───Y^Symbol("31")───Z^Symbol("32")───@───┼───
                                                                                                                                                                       │   │                                                      │   │
3: ───X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───@───X^Symbol("18")───Y^Symbol("19")───Z^Symbol("20")───@───X^Symbol("27")───Y^Symbol("28")───Z^Symbol("29")───X───┼───X^Symbol("33")───Y^Symbol("34")───Z^Symbol("35")───X───┼───
                                                         │                                                      │                                                          │                                                          │
4: ───X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───X───X^Symbol("21")───Y^Symbol("22")───Z^Symbol("23")───X───X^Symbol("36")───Y^Symbol("37")───Z^Symbol("38")───────@───X^Symbol("42")───Y^Symbol("43")───Z^Symbol("44")───────@───

This circuit contains 48 parameters (4 qubits x 2 "gates" / qubit x 6 parameters / gate). (Note that printing the circuit gets cut off in the notebook, scroll side to side to see the entire circuit.) For our simple example, we will chop off some of the gates to make the optimization easier:

In [11]:
# remove some of the gates and print it out
system.ansatz = system.ansatz[:-9]
system.ansatz

1: ───X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")────Y^Symbol("7")────Z^Symbol("8")────
                                                         │
2: ───X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")────Y^Symbol("10")───Z^Symbol("11")───

3: ───X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───@───X^Symbol("18")───Y^Symbol("19")───Z^Symbol("20")───
                                                         │
4: ───X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───X───X^Symbol("21")───Y^Symbol("22")───Z^Symbol("23")───

# Computing the Cost

The local cost function is computed via the Hadamard Test. The local cost function can be written
\begin{equation}
C_1 = 1 - \frac{1}{n} \sum_{k = 1}^{K} \sum_{l \geq k}^{K} w_{k, l} c_k c_l^* \sum_{j = 1}^{n} \text{Re} \, \langle V_{k, l}^{(j)} \rangle
\end{equation}
where
\begin{equation}
\langle V_{k, l}^{(j)} \rangle := \langle0^{\otimes n}| V^\dagger A_k^\dagger U P_j U^\dagger A_l V |0^{\otimes n}\rangle
\end{equation}
and 
\begin{equation}
    w_{k, l} = \begin{cases}
    1 \qquad \text{if } k = l\\
    2 \qquad \text{otherwise}
    \end{cases} .
\end{equation}
Thus we have $n K^2$ different circuits to compute the cost. An example of one (the $k = 0$, $l = 1$, $j = 1$ term) is shown below:

In [14]:
system.make_hadamard_test_circuit(system.ops[0], system.ops[1], 0, "real")

0: ───H─────────────────────────────────────────────────────────────────────────────────────────────────────────────@───@───@───@───────@───────@───@───@───H───M('z')───
                                                                                                                    │   │   │   │       │       │   │   │
1: ───────X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")────Y^Symbol("7")────Z^Symbol("8")────X───┼───┼───┼───X───@───X───Z───┼───┼────────────────
                                                             │                                                          │   │   │                   │   │
2: ───────X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")────Y^Symbol("10")───Z^Symbol("11")───────Y───┼───┼───X───────X───────X───┼────────────────
                                                                                                                            │   │                       │
3: ───────X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───@───X^Symbol("18")───Y^Symbol("19")───Z^Symbol("20")───────────X───┼───────────────────────┼────────────────
                                                             │                                                                  │                       │
4: ───────X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───X───X^Symbol("21")───Y^Symbol("22")───Z^Symbol("23")───────────────Z───X───────X───────────Y────────────────

The circuit for computing the norm
\begin{equation}
\langle 0 | V^\dagger A_k^\dagger A_l V | 0 \rangle = \langle \psi | A_k^\dagger A_l | \psi \rangle 
\end{equation}
for the example $k = 0$, $l = 1$ is shown below:

In [15]:
system.make_norm_circuit(system.ops[0], system.ops[1], "real")

0: ───H─────────────────────────────────────────────────────────────────────────────────────────────────────────────@───@───@───@───@───@───@───H───M('z')───
                                                                                                                    │   │   │   │   │   │   │
1: ───────X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")────Y^Symbol("7")────Z^Symbol("8")────X───┼───┼───┼───Z───┼───┼────────────────
                                                             │                                                          │   │   │       │   │
2: ───────X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")────Y^Symbol("10")───Z^Symbol("11")───────Y───┼───┼───────X───┼────────────────
                                                                                                                            │   │           │
3: ───────X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───@───X^Symbol("18")───Y^Symbol("19")───Z^Symbol("20")───────────X───┼───────────┼────────────────
                                                             │                                                                  │           │
4: ───────X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───X───X^Symbol("21")───Y^Symbol("22")───Z^Symbol("23")───────────────Z───────────Y────────────────

To compute the cost, we can call `PauliSystem.cost` or `PauliSystem.eff_cost` (the latter exploits symmetries to compute the cost more efficiently) and pass in a set of angles to the ansatz gates:

In [39]:
# =======================================
# compute the cost for some set of angles
# =======================================

# normalize the coefficients
system.normalize_coeffs()

# get some angles
angles = np.random.randn(24)

# compute the cost and time it
start = time.time()
cost = system.eff_cost(angles)
end = time.time() - start

# print out the results
print("Local cost C_1 =", cost)
print("Time to compute cost =", end, "seconds")

0.794123003933163
Local cost C_1 = 0.794123003933163
Time to compute cost = 0.9617917537689209 seconds


# Solving the System

To solve the system, we minimize the cost function. We'll do this below with the Powell optimization algorithm.

In [40]:
start = time.time()
out = system.solve(x0=angles, opt_method="Powell")
end = time.time() - start

0.7988090974336084
0.7926201548697706
0.8446648368853361
0.7765980976433634
0.8194725357953301
0.7904579241643355
0.8439249205903918
0.7761289579674315
0.7781456339729066
0.789796020607269
0.7844280964396952
0.7703577149300247
0.7730845837212049
0.781003022438578
0.7766487503896099
0.7757260777831662
0.8351808383155154
0.7787331850243817
0.7707819067959831
0.8302858490024243
0.7807147023986094
0.7721829470029259
0.7986271202645051
0.8037972841002705
0.7827717481789147
0.7718401921333787
0.7784265427371263
0.7795987884067515
0.7651838454640615
0.7842076485493441
0.7778977564633578
0.7772727087107957
0.7787285114030194
0.7759296533784705
0.7826729176774337
0.7802822400039497
0.7838370791366839
0.7766030635455052
0.7740059693123622
0.7691105503266911
0.7872649774142769
0.7707678981487214
0.7780315974555794
0.8015373718664396
0.7707576880641831
0.7768262262794703
0.770945583330255
0.7718930151183764
0.7670101550767043
0.7604292835231061
0.7691820441799484
0.763742515734624
1.20519153337029

0.6167850338246185
0.6508952444564613
0.6435942067729044
0.6122804623563497
0.6192775038076401
0.6189607712695593
0.6124463136480811
0.6147958443687127
0.6125389449402623
0.6160973593014012
0.6188817694560599
0.6172489792614722
0.6134675016693396
0.6130288533361437
0.6162053060453301
0.6123263752623518
1.3221732613736126
0.8488927665308187
0.6124546028017062
1.0902471142391403
0.8454877541720207
0.6114871996571123
0.615653611043189
0.7140702660650042
0.6403881127833396
0.6133928795084092
0.6185588721841999
0.6174214145671248
0.6207564647007059
0.6138197083069535
0.6151767811098118
0.6202645845628512
0.613561577008512
0.6224643782407916
0.6529052389951454
0.6325316070505633
0.6160644226569525
0.6486010841999217
0.6212258673944792
0.6137938649840209
0.6158122830083022
0.6084939422428751
0.6214820303070634
0.61490549177067
0.6154434516883438
0.613489022853692
0.6127367744648791
0.615998789203099
1.3181489928770147
0.8580701203456582
0.6119503273756911
1.061763986883223
0.8596078121197306


0.6049290435620409
0.8154644490925372
0.6948329530716676
0.6020764484665011
0.6522097577971013
0.6155136957886664
0.6062152346158578
0.6000396178003636
0.6019963772204535
0.611563492781088
0.6024569868316703
0.5948266944873328
0.6022069445131903
0.6555954107137832
0.6012778770215019
0.9295011081117399
0.6864814067373698
0.6034288990922945
0.8338058359710467
0.6855692267583915
0.6026741794067985
0.5975573309267372
0.6198923633536996
0.6049695133766712
0.6024897221766545
0.6070593937948348
0.6024928881523468
0.6031678886565546
0.5979178061157672
0.6020420428386664
0.6004842106962627
0.6021746095194648
0.6038845669833435
0.5964331021927929
0.8646970677023291
0.6962484892899757
0.6037909662022061
0.7761719434132694
0.6944991671969469
0.5972886800765773
0.6361836095452158
0.6174539032466794
0.5926417446039824
0.6017006115879973
0.5949814629629384
0.6006728647858846
0.5984762847179774
0.6062986361073532
0.6059843321632777
0.6075922950885195
0.6012702835685197
0.7005213680181275
0.63779096161

0.6025348126127342
0.6029974913552704
0.6023620770745644
0.6028488550095872
0.6040619603323236
0.6018904106834003
0.8042737250529418
0.6646175759229467
0.601503134178963
0.7362109469129277
0.6645484000102708
0.5984991373748796
0.6218707967350828
0.60500696657934
0.6016399889950899
0.6033792022424237
0.6076901494765424
0.5972595711993991
0.6049141651648409
0.6061819106008692
0.6063425746645595
0.607013912564031
0.5976436084219114
0.6027294359029796
0.6032959865943514
0.7627297794896089
0.6469999611037721
0.606406546905469
0.7122442789698018
0.6473704539838343
0.6086770779362842
0.6146933398492255
0.6090024018881774
0.6065791171973817
0.5991666015284864
0.596292519321834
0.6053331860785093
0.6077225656117302
0.6049437302365743
0.59777737218953
0.597671258951959
0.6073870840764795
0.9120998367038922
0.6856189585602401
0.5981679239507379
0.8227645993027077
0.6993316090228908
0.595235773756445
0.6594775683898999
0.6048544221780743
0.6053936665933085
0.6090893038687852
0.5982548702658906


In [47]:
print("It took {} minutes to solve the system.".format(round(end / 60)))
print("Number of iterations of optimization method:", out["nit"])
print("Number of function evaluations:", out["nfev"])

It took 21 minutes to solve the system.
Number of iterations of optimization method: 3
Number of function evaluations: 1357


In [48]:
# get the optimal angles
opt_angles = out["x"]

# get a param resolver
param_resolver = ParamResolver(
    {str(ii) : opt_angles[ii] for ii in range(len(opt_angles))}
)

sol_circ = system.ansatz.with_parameters_resolved_by(param_resolver)
sol_circ

1: ───X^-0.843───Y^-0.136───Z^-0.793───@───X^0.828───Y^-0.528───Z^0.448────
                                       │
2: ───X^0.412────Y^0.345────Z^-0.7─────X───X^0.414───Y^0.615────Z^-0.967───

3: ───X^0.494────Y^-0.479───Z^0.708────@───X^0.367───Y^0.348────Z^0.855────
                                       │
4: ───X^0.648────Y^0.842────Z^0.0682───X───X^0.118───Y^0.836────Z^0.461────

In [49]:
xhat = sol_circ.to_unitary_matrix()[:, 0]
bhat = np.dot(matrix, xhat)

In [50]:
print(bhat)
print(b)
print(np.linalg.norm(bhat - b))

[-1.19707873e-02-0.00716412j  1.23397682e-01-0.00516847j
  5.86568354e-02-0.11320653j  5.43591610e-04-0.01347711j
 -3.58960748e-02+0.10012684j -2.33992955e-01-1.10977592j
 -9.33437965e-01-0.27379139j -1.23033736e-01+0.01574063j
 -4.08314634e-04+0.0121113j  -4.53600336e-02-0.15656742j
 -1.10527492e-01+0.00227891j -1.75126061e-02+0.00359503j
 -1.06630601e-01-0.01790696j  1.23517522e+00-0.3116894j
  1.07954877e-01-0.98151758j -2.31789909e-02-0.13760881j]
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
1.867918837821211


# Future Work

* Normalize the cost to be between zero and one! (divide by N_Ax)
* Better optimization methods.
    * Optimize over a subset of the parameters at a time, then loop through (and reoptimize).
    * Add random gates using simulated annealing.
    * Compute all $nK^2$ circuits in parallel.
* Compute expectations of local observables at each cost iteration.
* Allow for arbitrary unitaries (not just Paulis)